In [1]:
import pandas as pd
train = pd.read_csv('C:\\Users\\saulpark\\Desktop\\scotch_review.csv')

train.shape

(2247, 7)

In [2]:
from bs4 import BeautifulSoup

example1 = BeautifulSoup(train['description'][0],"html5lib")
print(train['description'][0][:])
#아래
example1.get_text()[:]

Magnificently powerful and intense. Caramels, dried peats, elegant cigar smoke, seeds scraped from vanilla beans, brand new pencils, peppercorn, coriander seeds, and star anise make for a deeply satisfying nosing experience. Silky caramels, bountiful fruits of ripe peach, stewed apple, orange pith, and pervasive smoke with elements of burnt tobacco. An abiding finish of smoke, dry spices, and banoffee pie sweetness. Close to perfection. Editor's Choice


"Magnificently powerful and intense. Caramels, dried peats, elegant cigar smoke, seeds scraped from vanilla beans, brand new pencils, peppercorn, coriander seeds, and star anise make for a deeply satisfying nosing experience. Silky caramels, bountiful fruits of ripe peach, stewed apple, orange pith, and pervasive smoke with elements of burnt tobacco. An abiding finish of smoke, dry spices, and banoffee pie sweetness. Close to perfection. Editor's Choice"

In [3]:
import re

letters_only = re.sub('[^a-zA-Z]', ' ', example1.get_text())
letters_only[:700]

'Magnificently powerful and intense  Caramels  dried peats  elegant cigar smoke  seeds scraped from vanilla beans  brand new pencils  peppercorn  coriander seeds  and star anise make for a deeply satisfying nosing experience  Silky caramels  bountiful fruits of ripe peach  stewed apple  orange pith  and pervasive smoke with elements of burnt tobacco  An abiding finish of smoke  dry spices  and banoffee pie sweetness  Close to perfection  Editor s Choice'

In [4]:
lower_case = letters_only.lower()
words = lower_case.split()
print(len(words))
words[:10]

67


['magnificently',
 'powerful',
 'and',
 'intense',
 'caramels',
 'dried',
 'peats',
 'elegant',
 'cigar',
 'smoke']

In [5]:
import nltk
from nltk.corpus import stopwords
stopwords.words('english')[:10]

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're"]

In [6]:
# stopwords 를 제거한 토큰들
words = [w for w in words if not w in stopwords.words('english')]
print(len(words))
words[:10]

53


['magnificently',
 'powerful',
 'intense',
 'caramels',
 'dried',
 'peats',
 'elegant',
 'cigar',
 'smoke',
 'seeds']

In [7]:
# 랭커스터 스태머의 사용 예
from nltk.stem.lancaster import LancasterStemmer
lancaster_stemmer = LancasterStemmer()
print(lancaster_stemmer.stem('maximum'))
print("The stemmed form of running is: {}".format(lancaster_stemmer.stem("running")))
print("The stemmed form of runs is: {}".format(lancaster_stemmer.stem("runs")))
print("The stemmed form of run is: {}".format(lancaster_stemmer.stem("run")))

maxim
The stemmed form of running is: run
The stemmed form of runs is: run
The stemmed form of run is: run


In [8]:
# 처리 전 단어
words[:10]

['magnificently',
 'powerful',
 'intense',
 'caramels',
 'dried',
 'peats',
 'elegant',
 'cigar',
 'smoke',
 'seeds']

In [9]:
from nltk.stem.snowball import SnowballStemmer

stemmer = SnowballStemmer('english')
words = [stemmer.stem(w) for w in words]
# 처리 후 단어
words[:10]

['magnific',
 'power',
 'intens',
 'caramel',
 'dri',
 'peat',
 'eleg',
 'cigar',
 'smoke',
 'seed']

In [13]:
def review_to_words( raw_review ):
    # 1. HTML 제거
    review_text = BeautifulSoup(raw_review, 'html.parser').get_text()
    # 2. 영문자가 아닌 문자는 공백으로 변환
    letters_only = re.sub('[^a-zA-Z]', ' ', review_text)
    # 3. 소문자 변환
    words = letters_only.lower().split()
    # 4. 파이썬에서는 리스트보다 세트로 찾는게 훨씬 빠르다.
    # stopwords 를 세트로 변환한다.
    stops = set(stopwords.words('english'))
    # 5. Stopwords 불용어 제거
    meaningful_words = [w for w in words if not w in stops]
    # 6. 어간추출
    stemming_words = [stemmer.stem(w) for w in meaningful_words]
    # 7. 공백으로 구분된 문자열로 결합하여 결과를 반환
    return( ' '.join(stemming_words))

In [14]:
clean_review = review_to_words(train['description'][0])
clean_review

'magnific power intens caramel dri peat eleg cigar smoke seed scrape vanilla bean brand new pencil peppercorn coriand seed star anis make deepli satisfi nose experi silki caramel bounti fruit ripe peach stew appl orang pith pervas smoke element burnt tobacco abid finish smoke dri spice banoffe pie sweet close perfect editor choic'

In [15]:
num_reviews = train['description'].size
num_reviews

2247

In [16]:
from multiprocessing import Pool
import numpy as np

def _apply_df(args):
    df, func, kwargs = args
    return df.apply(func, **kwargs)

def apply_by_multiprocessing(df, func, **kwargs):
    # 키워드 항목 중 workers 파라메터를 꺼냄
    workers = kwargs.pop('workers')
    # 위에서 가져온 workers 수로 프로세스 풀을 정의
    pool = Pool(processes=workers)
    # 실행할 함수와 데이터프레임을 워커의 수 만큼 나눠 작업
    result = pool.map(_apply_df, [(d, func, kwargs)
            for d in np.array_split(df, workers)])
    pool.close()
    # 작업 결과를 합쳐서 반환
    return pd.concat(list(result))

In [ ]:
%time clean_train_reviews = apply_by_multiprocessing(\
    train['description'], review_to_words, workers=4)

In [ ]:
print("hi")